In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta

import warnings
warnings.filterwarnings("ignore")

In [2]:
def interval_split(dt):
    #for date_time column of "sXXdXXpXX"
    f2, f3 = dt.find("d"), dt.find("p")
    return [int(dt[1:f2]), int(dt[f2+1:f3]), int(dt[f3+1:])]

def date_split(dt):
    #for date_time column of "sXXdXX"
    f2 = dt.find("d")
    return [int(dt[1:f2]), int(dt[f2+1:])]


def add_interval(df):
    df_interval_data = np.vstack(df.date_time.apply(lambda x: interval_split(x)))
    df[["asset", "day", "interval"]] = df_interval_data
    return df

def add_date(df):
    df_date_data = np.vstack(df.date_time.apply(lambda x: date_split(x)))
    df[["asset", "day"]] = df_date_data
    return df

#Features in Consideration
def merge_columns(market, fundamental, returns=pd.DataFrame([0])):
    market = add_interval(market)
    fundamental = add_date(fundamental)
    df = pd.merge(market, fundamental, left_on=["asset", "day"], right_on=["asset", "day"])
    if returns.any()[0]:    #check if returns is a non-empty dataframe
        returns = add_date(returns)
        df = pd.merge(df, returns, left_on=["asset", "day"], right_on=["asset", "day"])
    # Note: This Step Will Mean that Metrics regarding
    return df

print("Ran")

Ran


# Paths and Load CSV (Modify Path Here)

In [3]:
path = '''second_round_datasets/'''
fn_train_fund = path + '''second_round_train_fundamental_data.csv'''
fn_train_market = path + '''second_round_train_market_data.csv'''
fn_train_returns = path + '''second_round_train_return_data.csv'''

df_train_fund = pd.read_csv(fn_train_fund)
df_train_market = pd.read_csv(fn_train_market)
df_train_returns = pd.read_csv(fn_train_returns)

df = merge_columns(df_train_market, df_train_fund, df_train_returns)
df

,date_time_x,open,close,high,low,volume,money,asset,day,interval,date_time_y,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf,date_time,return
0,s0d1p1,25.0164,24.9557,25.1378,24.9436,642243.0,1.610060e+07,0,1,1,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
1,s0d1p2,24.9557,24.8465,24.9557,24.8101,797872.0,1.984058e+07,0,1,2,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
2,s0d1p3,24.8465,24.7008,24.8586,24.6644,1234546.0,3.056567e+07,0,1,3,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
3,s0d1p4,24.7008,24.7615,24.7858,24.7008,533023.0,1.318848e+07,0,1,4,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
4,s0d1p5,24.7615,24.7251,24.7615,24.7008,565000.0,1.396949e+07,0,1,5,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4584595,s53d1698p46,16.0586,16.0100,16.0586,16.0100,257200.0,4.123711e+06,53,1698,46,s53d1698,0.8237,18198.0,14.5151,17.6623,2.1292,0.3268,13.2547,s53d1698,0.006065
4584596,s53d1698p47,15.9979,15.9979,16.0100,15.9736,181700.0,2.904803e+06,53,1698,47,s53d1698,0.8237,18198.0,14.5151,17.6623,2.1292,0.3268,13.2547,s53d1698,0.006065
4584597,s53d1698p48,15.9857,15.9857,16.0100,15.9736,163600.0,2.614642e+06,53,1698,48,s53d1698,0.8237,18198.0,14.5151,17.6623,2.1292,0.3268,13.2547,s53d1698,0.006065
4584598,s53d1698p49,15.9857,16.0222,16.0464,15.9857,145800.0,2.334710e+06,53,1698,49,s53d1698,0.8237,18198.0,14.5151,17.6623,2.1292,0.3268,13.2547,s53d1698,0.006065


In [5]:
#just a base to be implemented with for loop
asset_type = 19

df_a = df[df['asset'] == asset_type]
df_test = df_a.reset_index()
df_test.rename({'index':'old_index'}, axis=1, inplace=True)
df_test
#talib.py
df_test['%K'] = ta.momentum.stoch(df_test.high, df_test.low, df_test.close, window=14, smooth_window=3)
df_test['%D'] = df_test['%K'].rolling(3).mean()
df_test['RSI'] = ta.momentum.rsi(df_test.close, window=14)
df_test['MACD'] = ta.trend.macd_diff(df_test.close)
df_test


ImportError: cannot import name 'momentum' from 'talib' (/Users/pvpswaghd/Library/Python/3.8/lib/python/site-packages/talib/__init__.py)

In [ ]:
def gettrigger(df, lags, buy=True):
    dfx = pd.DataFrame()
    for i in range(1, 4):
        if buy:
            mask = (df_test['%K'].shift(i) < 20) & (df_test['%D'].shift(i) < 20)
        else:
            mask = (df_test['%K'].shift(i) > 80) & (df_test['%D'].shift(i) < 80)
        dfx = dfx.append(mask, ignore_index=True)
    return dfx.sum(axis=0)

def double_death_cross_stock(df, buy=True): 
    df['SMA_54'] = df['close'].rolling(window=54).mean()
    df['SMA_216'] = df['close'].rolling(window=216).mean()
    return df

def get_total_return(df, asset_type):
    df_a = df[df['asset'] == asset_type]
    df_test = df_a.reset_index()
    df_test.rename({'index':'old_index'}, axis=1, inplace=True)
    df_test
    df_test['%K'] = ta.momentum.stoch(df_test.high, df_test.low, df_test.close, window=14, smooth_window=3)
    df_test['%D'] = df_test['%K'].rolling(3).mean()
    df_test['RSI'] = ta.momentum.rsi(df_test.close, window=14)
    df_test['MACD'] = ta.trend.macd_diff(df_test.close)
    df_test['BuyTrigger'] = np.where(gettrigger(df, 4, True), 1, 0)
    #df_test['SellTrigger'] = np.where(gettrigger(df, 4, False), 1, 0)
    df_test['Buy'] = np.where((df_test['BuyTrigger']) & 
                            (df_test['%K'].between(20,80)) &
                            (df_test['%D'].between(20,80)) &
                            (df_test['RSI'] < 50) &
                            (df_test['MACD'] < 0), 1, 0)
    return df_test[(df_test['interval'] == 50) & (df_test['Buy'] == 1)]['return'].sum()

In [ ]:
sum = 0
for i in range(54):
    print(double_death_cross_stock(df))
    sum += get_total_return(df, i)


7.8772504525556055
